<a href="https://colab.research.google.com/github/Prakash-Nagamani/NLP_word_prediction/blob/main/DeepLearningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import string
import requests
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
def datacleaning(document):
  tokens = document.split()
  rem_punctuation=str.maketrans('','',string.punctuation)
  tokens=[w.translate(rem_punctuation) for w in tokens]
  tokens = [ w for w in tokens if w.isalpha()]
  tokens=[w.lower() for w in tokens]
  return tokens

##**Data Cleaning**

In [3]:
response = requests.get('https://www.gutenberg.org/cache/epub/5200/pg5200.txt')

In [4]:
data = response.text.split('\n')

In [5]:
data = data[46:]
data[0]

'One morning, when Gregor Samsa woke from troubled dreams, he found\r'

In [6]:
# len(data)

In [7]:
data = " ".join(data)

In [8]:
data


'One morning, when Gregor Samsa woke from troubled dreams, he found\r himself transformed in his bed into a horrible vermin.  He lay on\r his armour-like back, and if he lifted his head a little he could\r see his brown belly, slightly domed and divided by arches into stiff\r sections.  The bedding was hardly able to cover it and seemed ready\r to slide off any moment.  His many legs, pitifully thin compared\r with the size of the rest of him, waved about helplessly as he\r looked.\r \r "What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\r a proper human room although a little too small, lay peacefully\r between its four familiar walls.  A collection of textile samples\r lay spread out on the table - Samsa was a travelling salesman - and\r above it there hung a picture that he had recently cut out of an\r illustrated magazine and housed in a nice, gilded frame.  It showed\r a lady fitted out with a fur hat and fur boa who sat upright,\r raising a heavy fur muff that c

In [9]:
tokens = datacleaning(data)

In [10]:
len(set(tokens))

3020

In [11]:
sequence_len = 45+1
lines = []
for i in range(sequence_len,len(tokens)):
  seq = tokens[i-sequence_len:i]
  line = " ".join(seq)
  lines.append(line)


In [12]:
lines[0]

'one morning when gregor samsa woke from troubled dreams he found himself transformed in his bed into a horrible vermin he lay on his armourlike back and if he lifted his head a little he could see his brown belly slightly domed and divided by arches'

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
seq=tokenizer.texts_to_sequences(lines)

In [14]:
seq=np.array(seq)
seq

array([[  69,  161,   61, ..., 3007,   54, 1529],
       [ 161,   61,   15, ...,   54, 1529,   53],
       [  61,   15,  104, ..., 1529,   53,  821],
       ...,
       [ 819,   83,  537, ..., 3020,    2,  269],
       [  83,  537,    1, ...,    2,  269,   58],
       [ 537,    1, 1070, ...,  269,   58,  268]])

In [15]:
first,pred = seq[:,:-1],seq[:,-1]
first,pred

(array([[  69,  161,   61, ...,    3, 3007,   54],
        [ 161,   61,   15, ..., 3007,   54, 1529],
        [  61,   15,  104, ...,   54, 1529,   53],
        ...,
        [ 819,   83,  537, ...,  820, 3020,    2],
        [  83,  537,    1, ..., 3020,    2,  269],
        [ 537,    1, 1070, ...,    2,  269,   58]]),
 array([1529,   53,  821, ...,  269,   58,  268]))

In [16]:
# Index of each word
# tokenizer.word_index

In [17]:
vocabulary = len(tokenizer.word_index) + 1
# vocabulary=len(set(cleaneddata))+1
pred= to_categorical(pred,vocabulary)

In [18]:
input_len=first.shape[1]
input_len

45

##**Creating and Training Model**

In [19]:
model=Sequential()
model.add(Embedding(vocabulary,45,input_length=input_len))
model.add(LSTM(500,return_sequences=True))
model.add(LSTM(500))
model.add(Dense(500,activation='relu'))
model.add(Dense(vocabulary,activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 45)            135945    
                                                                 
 lstm (LSTM)                 (None, 45, 500)           1092000   
                                                                 
 lstm_1 (LSTM)               (None, 500)               2002000   
                                                                 
 dense (Dense)               (None, 500)               250500    
                                                                 
 dense_1 (Dense)             (None, 3021)              1513521   
                                                                 
Total params: 4,993,966
Trainable params: 4,993,966
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [22]:
model.fit(first,pred,batch_size=64,epochs=100)

Epoch 1/100
389/389 [==============================] - 34s 66ms/step - loss: 6.3524 - accuracy: 0.0517
Epoch 2/100
389/389 [==============================] - 26s 66ms/step - loss: 6.0001 - accuracy: 0.0564
Epoch 3/100
389/389 [==============================] - 26s 66ms/step - loss: 5.8256 - accuracy: 0.0647
Epoch 4/100
389/389 [==============================] - 26s 66ms/step - loss: 5.6914 - accuracy: 0.0709
Epoch 5/100
389/389 [==============================] - 26s 66ms/step - loss: 5.5755 - accuracy: 0.0761
Epoch 6/100
389/389 [==============================] - 26s 66ms/step - loss: 5.4536 - accuracy: 0.0895
Epoch 7/100
389/389 [==============================] - 26s 67ms/step - loss: 5.3266 - accuracy: 0.0972
Epoch 8/100
389/389 [==============================] - 26s 67ms/step - loss: 5.1964 - accuracy: 0.1077
Epoch 9/100
389/389 [==============================] - 26s 67ms/step - loss: 5.0686 - accuracy: 0.1163
Epoch 10/100
389/389 [==============================] - 26s 66ms/step - l

#Predicting next n words using the model

In [30]:
def generate_text_seq(model,tokenizer,input_len,seed_text,n):
  data=[]
  for i in range(n):
    encoded=tokenizer.texts_to_sequences([seed_text])[0]
    encoded=pad_sequences([encoded],maxlen=input_len,truncating='pre')
    encoded=np.array(encoded)
    y_predict=model.predict(encoded)
    m=np.argmax(y_predict,axis=1)
    predicted_word=""
    for word,index in tokenizer.word_index.items():
      if index==m[0]:
        predicted_word=word
        break
    seed_text=seed_text+" "+predicted_word
    data.append(predicted_word)
  return " ".join(data)

In [33]:
seed_text= "And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual"
n=int(input("Enter the no. of words you want to predict: "))
generate_text_seq(model,tokenizer,input_len,seed_text,n)

Enter the no. of words you want to predict: 20


'he could think of taking it and looking himself but he could think of him and with course the three'